In [6]:
from pprint import pprint as pp
import pandas as pd
import json
import csv
import re

In [11]:
class TimeParser:
    __no_break = ['No Break time', 'Break time 없음', '쉬는시간 없음', '브레이크타임없음', '브레이크타임 없음', 'B.T 없음', 'BT없음']
    __breaktime = [r'B.T:?\d{2}[-~]\d{2}',
                   r'b[a-zA-Z]{4}t[a-zA-Z]{3}:?\d{2}[-~]\d{2}',
                   r'[bB]{1}[a-zA-z]{4}:?\d{2}:\d{2}[-~]\d{2}:\d{2}',
                   r'[bB]{1}[a-zA-Z]{4}[tT]{1}[a-zA-Z]{3}:?\d{2}:\d{2}[-~]\d{2}:\d{2}',
                   r'브레이크타임:?\d{2}[-~]\d{2}',
                   r'브레이크타임:?\d{2}:\d{2}[-~]\d{2}:\d{2}',
                   r'준비시간:?\d{2}:\d{2}[-~]\d{2}:\d{2}',
                   r'쉬는시간:?\d{2}:\d{2}[-~]\d{2}:\d{2}',
                   r'브레이크타임\d{2}시[-~]\d{2}시',
                   r'[bB]{1}[a-zA-Z]{4}[tT]{1}[a-zA-Z]{3}\d{2}시[-~]\d{2}시',
                   r'\d{2}:\d{2}[-~]\d{2}:\d{2}브레이크타임',
                   r'\d{2}:\d{2}[-~]\d{2}:\d{2}[bB]{1}[a-zA-Z]{4}[tT]{1}[a-zA-Z]{3}',
                   r'\d{2}:\d{2}[-~]\d{2}:\d{2}휴게시간']

    __day_break = [r'^Breaktime$', r'^BreakTime$', r'^breaktime$', r'^BREAKTIME$', r'^점심시간$', r'^휴식시간$', r'^브레이크타임$', r'^휴식시간(BreakTime)$']
    __lastorder = [r'L.O\d{2}:\d{2}', r'lastorder\d{2}:\d{2}',
                   r'Lastorder:?\d{2}:\d{2}', r'LastOrder:?\d{2}:\d{2}',
                   r'LASTORDER:?\d{2}:\d{2}', r'LastOrder오후\d{1,2}시',
                   r'라스트오더\d{2}:\d{2}', r'주문마감\d{2}:\d{2}', 
                   r'마지막입장\d{2}:\d{2}', r'마지막주문:?\d{2}:\d{2}',
                   r'LastOrderPM\d{2}:\d{2}', r'LastOrder오후\d{2}:\d{2}']
    
    def __setting_init_store(self, store_id, businessHours):
        self.__businessHours = eval(businessHours)
        self.__store_id = store_id
        self.__BusinessHour = {
            "월요일" : { "store": store_id, "day": "월요일", "is_dayoff": False, "start_time": "00:00", "end_time": "23:59", "start_break": None, "end_break": None, "last_order": None, "memo": None },
            "화요일" : { "store": store_id, "day": "화요일", "is_dayoff": False, "start_time": "00:00", "end_time": "23:59", "start_break": None, "end_break": None, "last_order": None, "memo": None },
            "수요일" : { "store": store_id, "day": "수요일", "is_dayoff": False, "start_time": "00:00", "end_time": "23:59", "start_break": None, "end_break": None, "last_order": None, "memo": None },
            "목요일" : { "store": store_id, "day": "목요일", "is_dayoff": False, "start_time": "00:00", "end_time": "23:59", "start_break": None, "end_break": None, "last_order": None, "memo": None },
            "금요일" : { "store": store_id, "day": "금요일", "is_dayoff": False, "start_time": "00:00", "end_time": "23:59", "start_break": None, "end_break": None, "last_order": None, "memo": None },
            "토요일" : { "store": store_id, "day": "토요일", "is_dayoff": False, "start_time": "00:00", "end_time": "23:59", "start_break": None, "end_break": None, "last_order": None, "memo": None },
            "일요일" : { "store": store_id, "day": "일요일", "is_dayoff": False, "start_time": "00:00", "end_time": "23:59", "start_break": None, "end_break": None, "last_order": None, "memo": None }
        }
        
        self.__HolidayHour = {
            "설날1" : { "store": store_id, "day": "설날1", "is_dayoff": False, "start_time": "00:00", "end_time": "23:59", "start_break": None, "end_break": None, "last_order": None, "memo": None },
            "설날2" : { "store": store_id, "day": "설날2", "is_dayoff": False, "start_time": "00:00", "end_time": "23:59", "start_break": None, "end_break": None, "last_order": None, "memo": None },
            "설날3" : { "store": store_id, "day": "설날3", "is_dayoff": False, "start_time": "00:00", "end_time": "23:59", "start_break": None, "end_break": None, "last_order": None, "memo": None },
            "추석1" : { "store": store_id, "day": "추석1", "is_dayoff": False, "start_time": "00:00", "end_time": "23:59", "start_break": None, "end_break": None, "last_order": None, "memo": None },
            "추석2" : { "store": store_id, "day": "추석2", "is_dayoff": False, "start_time": "00:00", "end_time": "23:59", "start_break": None, "end_break": None, "last_order": None, "memo": None },
            "추석3" : { "store": store_id, "day": "추석3", "is_dayoff": False, "start_time": "00:00", "end_time": "23:59", "start_break": None, "end_break": None, "last_order": None, "memo": None },
            "나머지" : { "store": store_id, "day": "나머지", "is_dayoff": False, "start_time": "00:00", "end_time": "23:59", "start_break": None, "end_break": None, "last_order": None, "memo": None }
        } 
    
    
    def __setting_dayoff(self, day_key):
        return { "store": self.__store_id, "day": day_key, "is_dayoff": True, "start_time": None, "end_time": None, "start_break": None, "end_break": None, "last_order": None, "memo": None }
    
    def __find_day_offs(self, dsc):
        days = []
        result = []
        if "주일" in dsc or "주말" in dsc or "월" in dsc or "화" in dsc or "수" in dsc or "목" in dsc or "금" in dsc or "토" in dsc or "일요일" in dsc or re.match(r'.* 일.*', dsc) or re.match(r'.*,일.*', dsc) or re.match(r'.*,일?일.*', dsc) or re.match(r'.*,일요.*', dsc):
            days = re.findall(r'[\w]{1}요일', dsc)
        for day in days:
            if day == "월요일" or day == "화요일" or day == "수요일" or day == "목요일" or day =="금요일" or day =="토요일" or day =="일요일":
                result.append(day)
        if "월" in dsc:
            if not "회" in dsc and not "매월" in dsc:
                result.append("월요일")
        if "화" in dsc:
            if not "백화점" in dsc and not "전화" in dsc:
                result.append("화요일")
        if "수" in dsc:
            if not "홀수" in dsc and not "짝수" in dsc and not "수업" in dsc and not "수 있" in dsc and not "수있" in dsc and not "이수" in dsc and not "필수" in dsc and not "수족관" in dsc and not "가로수길" in dsc and not "예수" in dsc and not "수시" in dsc and not "수급" in dsc and not "특수" in dsc:
                result.append("수요일")
        if "목" in dsc:
            result.append("목요일")
        if "금" in dsc:
            if not "(금)" in dsc and not "금일" in dsc and not "요금" in dsc and not "조금" in dsc and not "앙금" in dsc:
                result.append("금요일")
        if "토" in dsc:
            result.append("토요일")
        if re.match(r'.* 일.*', dsc) or re.match(r'.*,일.*', dsc) or "주일" in dsc:
            if not "일반" in dsc and not "일부" in dsc and not "일주일" in dsc and not "일찍" in dsc and not "일하" in dsc and not "일정" in dsc:
                result.append("일요일")
        if re.search(r'주말.?휴', dsc):
            result.append("토요일")
            result.append("일요일")
        return set(result)

    def __find_holiday_offs(self, dsc):
        all_holiday_off = False
        legal_holidays_off = []
        if "명절" in dsc or "공휴" in dsc or "추석" in dsc or "설" in dsc or "구정" in dsc or "신정" in dsc: 
            if "명절" in dsc or "추석" in dsc or "설" in dsc or "구정" in dsc or "신정" in dsc: 
                if not "공휴일" in dsc:
                    if "당일" in dsc:
                        legal_holidays_off = ["설날2", "추석2"]
                    else:
                        legal_holidays_off = ["설날1", "설날2", "설날3", "추석1", "추석2", "추석3"]
                else:
                    all_holiday_off = True
            else:
                all_holiday_off = True
        else:
            return set([])

        if all_holiday_off:
            return set(["설날1", "설날2", "설날3", "추석1", "추석2", "추석3", "나머지"])

        else:
            return set(legal_holidays_off)
    
    def __find_total_offs(self, dsc):
        if "연중무휴" in dsc or "무휴" in dsc or "휴무없음" in dsc or "휴무일 없음" in dsc or "휴일없음" in dsc or "휴무일없움" in dsc or "연중휴무" in dsc or "휴무일없음" in dsc or "연중뮤휴" in dsc: 
            if "제외" in dsc or "예외" in dsc or "만" in dsc or "휴무" in dsc or "쉽" in dsc:
                if not "연중휴무" in dsc and not "휴무없음" in dsc and not "휴무일 없음" in dsc and not "무휴무" in dsc:
                    dayoffs = {}
                    dayoffs["BusinessHour"] = self.__find_day_offs(dsc)
                    dayoffs["HolidayHour"] = self.__find_holiday_offs(dsc)
                    return dayoffs
                return None

        if "휴" in dsc or "휴무" in dsc or "쉽" in dsc or "쉼" in dsc or "휴뮤" in dsc:
            if not "휴식" in dsc:
                dayoffs = {
                    "BusinessHour": [], "HolidayHour": []
                }
                dayoffs["BusinessHour"] = self.__find_day_offs(dsc)
                dayoffs["HolidayHour"] = self.__find_holiday_offs(dsc)
                return dayoffs

            else:
                return None

        else:
            return None
        
    def __find_weekend_offs(self, dsc):
        result = []
        offs = {"금요일": False, "토요일": False , "일요일": False }
        if not dsc:
            return result
        if "금" in dsc:
            if not "(금)" in dsc and not "금일" in dsc and not "요금" in dsc and not "조금" in dsc and not "앙금" in dsc:
                offs["금요일"] = True
            else:
                offs["금요일"] = False
        else:
            offs["금요일"] = False
        if "일요일 휴무" in dsc or "일요일휴무" in dsc:
            offs["일요일"] = True
        if "토요일 휴무" in dsc or "토요일휴무" in dsc:
            offs["토요일"] = True
        for key in offs:
            if offs[key]:
                result.append(key)
        
        return set(result)  
        
    def __parsing_days(self):
        
        days = { "매일": [], "평일": [], "주말": [], "공휴일": [], "월요일": [], "화요일": [], "수요일": [], "목요일": [], "금요일": [], "토요일": [], "일요일": [] }
        
        for table in self.__businessHours:
            days[table["day"]].append(table)
        
        
        # 중복 day 들이 존재할 수 있으니 우선 day 정리
        for key in days:
            day_info = { "is_dayoff": False, "start_time":  "24:00", "end_time": "00:00", "descriptions": [], "memo": "" }

            if days[key] == []:
                days[key] = None
                continue

            for day in days[key]:
                if day["isDayOff"] == True:
                    day_info["is_dayoff"] = True
                    break

                if day["startTime"] and day_info["start_time"]:
                    if day["startTime"] < day_info["start_time"]:
                        day_info["start_time"] = day["startTime"]

                if day["endTime"] and day_info["end_time"]:
                    if day["endTime"] > day_info["end_time"]:
                        day_info["end_time"] = day["endTime"]

                if day["description"]:
                    day_info["memo"] += day["description"] + " "
                    day_info["descriptions"].append(day["description"])

            days[key] = day_info

        # 이제 하나씩 조지기
        ## 매일
        if not days["매일"]:
            pass

        else:
            if days["매일"]["is_dayoff"]:
                pass

            else:
                biz_offs = set()
                holi_offs = set()
                for dsc in days["매일"]["descriptions"]:
                    day_offs = self.__find_total_offs(dsc)
                    if day_offs:
                        biz_offs = biz_offs | day_offs["BusinessHour"]
                        holi_offs = holi_offs | day_offs["HolidayHour"]
                if biz_offs:
                    for day_key in biz_offs:
                        self.__BusinessHour[day_key] = self.__setting_dayoff(day_key)
                if holi_offs:
                    holi_offs = holi_offs & {"설날1", "설날2", "설날3", "추석1", "추석2", "추석3", "나머지"}
                    for day_key in holi_offs:
                        self.__HolidayHour[day_key] = self.__setting_dayoff(day_key)

                for key in self.__BusinessHour:
                    if self.__BusinessHour[key]["is_dayoff"]:
                        continue
                    else:
                        self.__BusinessHour[key]["start_time"] = days["매일"]["start_time"]
                        self.__BusinessHour[key]["end_time"] = days["매일"]["end_time"]
                        self.__BusinessHour[key]["memo"] = days["매일"]["memo"]

                for key in self.__HolidayHour:
                    if self.__HolidayHour[key]["is_dayoff"]:
                        continue
                    else:
                        self.__HolidayHour[key]["start_time"] = days["매일"]["start_time"]
                        self.__HolidayHour[key]["end_time"] = days["매일"]["end_time"]
                        self.__HolidayHour[key]["memo"] = days["매일"]["memo"]
                        
        ## 평일
        if not days["평일"]:
            pass
        else:
            if days["평일"]["is_dayoff"]:
                pass
            else:
                biz_offs = set()
                holi_offs = set()

                for dsc in days["평일"]["descriptions"]:
                    day_offs = self.__find_total_offs(dsc)
                    if day_offs:
                        biz_offs = biz_offs | day_offs["BusinessHour"]
                        holi_offs = holi_offs | day_offs["HolidayHour"]
                if biz_offs:
                    for day_key in biz_offs:
                        self.__BusinessHour[day_key] = self.__setting_dayoff(day_key)

                if holi_offs:
                    holi_offs = holi_offs & {"설날1", "설날2", "설날3", "추석1", "추석2", "추석3", "나머지"}
                    for day_key in holi_offs:
                        self.__HolidayHour[day_key] = self.__setting_dayoff(day_key)

                for key in self.__BusinessHour:
                    if self.__BusinessHour[key]["is_dayoff"]:
                        continue
                    else:
                        self.__BusinessHour[key]["start_time"] = days["평일"]["start_time"]
                        self.__BusinessHour[key]["end_time"] = days["평일"]["end_time"]
                        self.__BusinessHour[key]["memo"] = days["평일"]["memo"]
                for key in self.__HolidayHour:
                    if self.__HolidayHour[key]["is_dayoff"]:
                        continue
                    else:
                        self.__HolidayHour[key]["start_time"] = days["평일"]["start_time"]
                        self.__HolidayHour[key]["end_time"] = days["평일"]["end_time"]
                        self.__HolidayHour[key]["memo"] = days["평일"]["memo"]
                        
        ## 주말
        if not days["주말"]:
            pass

        else:
            if days["주말"]["is_dayoff"]:
                self.__BusinessHour["토요일"] = self.__setting_dayoff("토요일")
                self.__BusinessHour["일요일"] = self.__setting_dayoff("일요일")
            else:
                weekend_offs = set()
                for dsc in days["주말"]["descriptions"]:
                    offs = self.__find_weekend_offs(dsc)
                    if offs:
                        weekend_offs = weekend_offs | offs   
                for day_key in ["금요일", "토요일", "일요일"]:
                    if day_key in weekend_offs:
                        self.__BusinessHour[day_key] = self.__setting_dayoff(day_key)
                    else:
                        self.__BusinessHour[day_key]["start_time"] = days["주말"]["start_time"]
                        self.__BusinessHour[day_key]["end_time"] = days["주말"]["end_time"]
                        self.__BusinessHour[day_key]["memo"] = days["주말"]["memo"]
        ## 공휴일
        if not days["공휴일"]:
            pass
        else:
            if days["공휴일"]["is_dayoff"]:
                for day_key in ["설날1", "설날2", "설날3", "추석1", "추석2", "추석3", "나머지"]:
                    self.__HolidayHour[day_key] = self.__setting_dayoff(day_key)
            else:
                holi_offs = set()

                for dsc in days["공휴일"]["descriptions"]:
                    day_offs = self.__find_total_offs(dsc)
                    if day_offs:
                        holi_offs = holi_offs | day_offs["HolidayHour"]
                if holi_offs:
                    holi_offs = holi_offs & {"설날1", "설날2", "설날3", "추석1", "추석2", "추석3", "나머지"}
                    for day_key in holi_offs:
                        self.__HolidayHour[day_key] = self.__setting_dayoff(day_key)

                for key in self.__HolidayHour:
                    if self.__HolidayHour[key]["is_dayoff"]:
                        continue
                    else:
                        self.__HolidayHour[key]["start_time"] = days["공휴일"]["start_time"]
                        self.__HolidayHour[key]["end_time"] = days["공휴일"]["end_time"]
                        self.__HolidayHour[key]["memo"] = days["공휴일"]["memo"]
        
        ## 요일별
        for day_key in ["월요일", "화요일", "수요일", "목요일", "금요일", "토요일", "일요일"]:
            if days[day_key]:
                if days[day_key]["is_dayoff"]:
                    self.__BusinessHour[day_key] = self.__setting_dayoff(day_key)
                else:
                    self.__BusinessHour[day_key]["start_time"] = days[day_key]["start_time"]
                    self.__BusinessHour[day_key]["end_time"] = days[day_key]["end_time"]
                    self.__BusinessHour[day_key]["memo"] = days[day_key]["memo"]
    
    def __parsing_times(self):
        for bu in self.__businessHours:
            bt = ''
            bs = ''
            be = ''
            if not bu:
                continue
                
            # 브레이크타임 찾기
            descript = bu.get("description")

            if not descript:
                continue
            descript = descript.replace(" ", "")

            # 브레이크타임 없는 경우
            for nb in self.__no_break:
                if nb in descript:
                    continue
                    
            day_bt = bu["day"]
            
            # 브레이크타임이 description에 있는 경우(시간정보 포함)
            for bk in self.__breaktime:
                if re.findall(bk, descript):
                    bt = re.findall(bk, descript)[0]
                    if bt[2] == ":":
                        bs = bt[:5]
                        be = bt[6:11]
                    elif bt[-3] == ":":
                        bs = bt[-11:-6]
                        be = bt[-5:]
                    elif bt[-1] == "시":
                        bs = bt[-7:-5] + ":00"
                        be = bt[-3:-1] + ":00"
                    else:
                        bs = bt[-5:-3] + ":00"
                        be = bt[-2:] + ":00"


                    if day_bt == "매일":
                        for d_key, d_value in self.__BusinessHour.items():
                            if not d_value["is_dayoff"]:
                                d_value["start_break"] = bs
                                d_value["end_break"] = be
                        for h_key, h_value in self.__HolidayHour.items():
                            if not h_value["is_dayoff"]:
                                h_value["start_break"] = bs
                                h_value["end_break"] = be
                    elif day_bt == "평일":
                        for d_key, d_value in self.__BusinessHour.items():
                            if d_key in ["월요일", "화요일", "수요일", "목요일", "금요일"] and not d_value["is_dayoff"]:
                                d_value["start_break"] = bs
                                d_value["end_break"] = be
                    elif day_bt == "주말":
                        for d_key, d_value in self.__BusinessHour.items():
                            if d_key in ["금요일", "토요일", "일요일"] and not d_value["is_dayoff"]:
                                d_value["start_break"] = bs
                                d_value["end_break"] = be
                    else:
                        if day_bt != "공휴일" and not self.__BusinessHour[day_bt]["is_dayoff"]:
                            self.__BusinessHour[day_bt]["start_break"] = bs
                            self.__BusinessHour[day_bt]["end_break"] = be
                        else:
                            for h_key, h_value in self.__HolidayHour.items():
                                if not h_value["is_dayoff"]:
                                    h_value["start_break"] = bs
                                    h_value["end_break"] = be
                    continue
                
            # 브레이크타임이 요일정보에 들어가있는 경우
            for db in self.__day_break:
                if re.findall(db, descript):
                    day_bs = bu["startTime"]
                    day_be = bu["endTime"]

                    if day_bt == "매일":
                        for d_key, d_value in self.__BusinessHour.items():
                            if not d_value["is_dayoff"]:
                                d_value["start_break"] = day_bs
                                d_value["end_break"] = day_be
                        for h_key, h_value in self.__HolidayHour.items():
                            if not h_value["is_dayoff"]:
                                h_value["start_break"] = day_bs
                                h_value["end_break"] = day_be
                    elif day_bt == "평일":
                        for d_key, d_value in self.__BusinessHour.items():
                            if d_key in ["월요일", "화요일", "수요일", "목요일", "금요일"] and not d_value["is_dayoff"]:
                                d_value["start_break"] = day_bs
                                d_value["end_break"] = day_be
                    elif day_bt == "주말":
                        for d_key, d_value in self.__BusinessHour.items():
                            if d_key in ["금요일", "토요일", "일요일"] and not d_value["is_dayoff"]:
                                d_value["start_break"] = day_bs
                                d_value["end_break"] = day_be
                    else:
                        if day_bt != "공휴일" and not self.__BusinessHour[day_bt]["is_dayoff"]:
                            self.__BusinessHour[day_bt]["start_break"] = day_bs
                            self.__BusinessHour[day_bt]["end_break"] = day_be
                        else:
                            for h_day, h_value in self.__HolidayHour.items():
                                if not h_value["is_dayoff"]:
                                    h_value["start_break"] = day_bs
                                    h_value["end_break"] = day_bs
                    continue

            # last order 찾기
            lo = ''
            for l in self.__lastorder:
                if re.findall(l, descript):
                    lo = re.findall(l, descript)[-1]

                    if lo[-1] == "시":
                        if lo[-3] == "후":
                            lo = str(int(lo[-2]) + 12) + ":00"
                        elif lo[-4] == "후":
                            lo = str(int(lo[-3:-1]) + 12) + ":00"
                        else:
                            lo = lo[-3:-1] + ":00" 
                    else:
                        lo = lo[-5:]

                    if day_bt == "매일":
                        for d_key, d_value in self.__BusinessHour.items():
                            if not d_value["is_dayoff"]:
                                d_value["last_order"] = lo
                        for h_key, h_value in self.__HolidayHour.items():
                            if not h_value["is_dayoff"]:
                                h_value["last_order"] = lo
                    elif day_bt == "평일":
                        for d_key, d_value in self.__BusinessHour.items():
                            if d_key in ["월요일", "화요일", "수요일", "목요일", "금요일"] and not d_value["is_dayoff"]:
                                d_value["last_order"] = lo
                    elif day_bt == "주말":
                        for d_key, d_value in self.__BusinessHour.items():
                            if d_key in ["금요일", "토요일", "일요일"] and not d_value["is_dayoff"]:
                                d_value["last_order"] = lo
                    else:
                        if day_bt != "공휴일" and not self.__BusinessHour[day_bt]["is_dayoff"]:
                            self.__BusinessHour[day_bt]["last_order"] = lo
                        else:
                            for h_day, h_value in self.__HolidayHour.items():
                                if not h_value["is_dayoff"]:
                                    h_value["last_order"] = lo
                    continue
        
        
    def __check_format(self):
        # TODO
        ## 만약 s_t, e_t 이 00:00~23:59 로 잡혀 있다면 is_dayoff를 켜야겠다.
        ## 그 외에 fasle 같은거 시간에 들어가 있는지 확인하고 제거
        return
    
    
    def __save_store(self):

        bizz = self.filename + "_BusinessHour"
        bizz_fields = ["store", "day", "is_dayoff", "start_time", "end_time", "start_break", "end_break", "last_order", "memo"]
        holii = self.filename + "_HolidayHour"
        holii_fields = ["store", "day", "is_dayoff", "start_time", "end_time", "start_break", "end_break", "last_order", "memo"]
        
        if self.__cnt == 0:    
            with open(bizz + ".csv", "w", newline="", encoding="utf8") as f:
                dictwriter = csv.DictWriter(f, fieldnames = bizz_fields)
                dictwriter.writeheader()
                for day in self.__BusinessHour:
                    dictwriter.writerow(self.__BusinessHour[day])
            with open(holii + ".csv", "w", newline="", encoding="utf8") as f:
                dictwriter = csv.DictWriter(f, fieldnames = holii_fields)
                dictwriter.writeheader()
                for day in self.__HolidayHour:
                    dictwriter.writerow(self.__HolidayHour[day])
            self.__cnt += 1

        else:
            with open(bizz + ".csv", "a", newline="", encoding="utf8") as f:
                dictwriter = csv.DictWriter(f, fieldnames = bizz_fields)
                for day in self.__BusinessHour:
                    dictwriter.writerow(self.__BusinessHour[day])
            with open(holii + ".csv", "a", newline="", encoding="utf8") as f:
                dictwriter = csv.DictWriter(f, fieldnames = holii_fields)
                for day in self.__HolidayHour:
                    dictwriter.writerow(self.__HolidayHour[day])
            self.__cnt += 1
    
    
    def __parsing_store(self, store_id, businessHours):
    
        if type(businessHours) == type(1.00002):
            return None
            
        # 가게별 초기화
        self.__setting_init_store(store_id, businessHours)
        
        # 날짜 파싱
        self.__parsing_days()
        
        # 시간 파싱
        self.__parsing_times()
        
        # 체크
        self.__check_format()
        
        # 저장
        self.__save_store()
    
    
    
    def __init__(self, filename): # filename은 확장자 떼고. 근데 무조건 csv 파일
        self.filename = str(filename)
        self.__cnt = 0
        
    def start_parsing(self):
        df_stores = pd.read_csv(self.filename+".csv", sep=",", encoding="utf8", engine="python")
        df_stores.apply(lambda row: self.__parsing_store(row["id"], row["businessHours"]), axis=1)    

In [13]:
TimeParser("gangnam_stores").start_parsing()